In [2]:
from transformers import pipeline, AutoModelForMaskedLM, AutoTokenizer, AutoConfig, AdapterConfig

In [12]:
model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [6]:
pipe = pipeline("fill-mask", model=model, tokenizer=tokenizer, top_k=5)

In [11]:
"AdsdSASas".lower()

'adsdsasas'

In [9]:
pipe("It was a <mask> day")

[{'sequence': 'It was a beautiful day',
  'score': 0.2355542778968811,
  'token': 2721,
  'token_str': ' beautiful'},
 {'sequence': 'It was a good day',
  'score': 0.10204992443323135,
  'token': 205,
  'token_str': ' good'},
 {'sequence': 'It was a great day',
  'score': 0.05954313278198242,
  'token': 372,
  'token_str': ' great'},
 {'sequence': 'It was a nice day',
  'score': 0.051886625587940216,
  'token': 2579,
  'token_str': ' nice'},
 {'sequence': 'It was a long day',
  'score': 0.03153948113322258,
  'token': 251,
  'token_str': ' long'}]

In [15]:
adapter_config = AdapterConfig.load("houlsby", non_linearity="gelu", reduction_factor=12)
model.add_adapter("test_sondre", config=adapter_config)

In [9]:
model.active_adapters = None

In [20]:
111893562-109514298

2379264

In [21]:
2379264/111893562

0.021263636240304872

In [16]:
sum(p.numel() for p in model.parameters())

111893562

In [14]:
sum(p.numel() for p in model.parameters())

109514298

In [6]:
model.config.adapters.adapters

{'test_sondre': '5792d51e3161642d'}

In [ ]:
model.train_adapter(["test_sondre"])

In [ ]:
model.set_active_adapters(["test_sondre"])

In [ ]:
model.freeze_model(False)

In [ ]:
model

In [ ]:
sum(p.numel() for p in model.parameters())

In [ ]:
for param in model.base_model.encoder.layer[2].output.adapters.parameters():
    i = 0
    while i < 10:
        i += 1
        print(param.requires_grad)
    